In [ ]:
##%matplotlib notebook
from ipywidgets import interact, widgets
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import optimize as opt

# Linear Regression and the Estimation Problem

It is often the case in scientific research that we need to model (or recover) the relationship between two variables, $Y$ and $X$, under noise contamination. If the relationship is linear, and we can postulate the additive model 
$$Y_i = \beta X_i + \epsilon_i ,$$ 
it is well-known that **linear regression** provides a solution that is often (under appropriate conditions) optimal. In statistics and econometrics, $\beta$ is called the **coefficient** (it may be a vector or matrix), while $\epsilon$ is the **noise**. 

If $X$ is a fixed matrix of *covariates* (also, *features* or *regressors*, depending on the whether we feel most comfortable with the ML or statistical nomenclature, respectively) - that is, we are fine with treating its entries as non-random - and $Y$ is determined by the above linear relationship, it is often the case that one has direct access to both and wants to recover $\beta$. But in the remainder of this section, we shall consider $X$ to also be randomly drawn from some distribution, as it is often the case in the real world.

**Example:** Think of $X = (X_1, X_2, \ldots, X_n) \in \mathbb{R}^n$ for $n \geq 1$ as the intensity of sunlight shining on a given city at midday, $Y = (Y_1, Y_2, \ldots, Y_n) \in \mathbb{R}^n$ the temperatue in the same city measured at the same time as the light intersity. Therefore, we take $n$ touples of observations $(X_i, Y_i)$ and, to be sure, we choose a random roster of cities all over the world. We think that the *sulight has a linear effect on temperature* and that only additive noise is present in our measurements. Model $Y_i = \beta X_i + \epsilon_i$ is thus valid.


We may then note, an this is simple to derive, that the best linear (unbiased) estimator of $\beta$ in terms of *squared loss* is the **least-squares estimator**, obtained by minimizing
$$ \text{arg}\min_{b \in \mathbb{R}} \sum_{i=1}^n ( Y_i - b X_i )^2 , $$
and has closed-form
$$ \widehat{\beta} := (X' X)^{-1} X' Y. $$

Below you can find a simple example of a linear regression (with an intercept):

In [ ]:
# PLOT: linear regression
def linreg_plot(n = 10):
    # generate data
    rng = np.random.default_rng(seed=12345)
    X = rng.uniform(low=0, high=4, size=n)
    E = rng.normal(loc=0, scale=0.3, size=n)
    beta = 0.7
    Y = -0.2 + beta * X + E

    # regression
    Z = np.column_stack([np.ones(n), X])
    beta_hat = np.linalg.solve(np.dot(Z.T, Z), np.dot(Z.T, Y))
    #print(beta_hat)

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(np.linspace(0, 4), (-0.2 + beta * np.linspace(0, 4)), c="0.6", zorder=1)
    ax.scatter(X, Y, zorder=2)
    ax.plot(np.linspace(0, 4), (beta_hat[0] + beta_hat[1] * np.linspace(0, 4)), c="C1", zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_xlim([-0.1, 4.1])
    ax.set_ylim([-1, 3.5])
    ax.set_title("Linear Regression")
    ax.legend(["True Line", "Data", "Least-squares Estimate"])
    fig.canvas.draw()

interact(linreg_plot, n=(3, 100));

## Nonlinear Least Squares and Random-Weights Neural Networks

In this first section, we are going to explore the problem of **nonlinear regression**.

Suppose now, unlike in the case of linear regression, that the relationship in our data is given by
$$ Y_i = f(X_i) + \epsilon_i ,$$
where $f(\cdot)$ is a smooth but otherwise unknown nonlinear function. What can we do here?

A classical statistical approach is to first think of a reasonable *class of functions* to which $f$ may belong. A straightforward guess would be *polynomials*: this is a somewhat basic guess because, if anything, we can think of the Taylor expasion of $f$ take somewhere within the domain of $X_i$. Polynomials are a reasonable first guess!

Let us say that $f(x) = a + b x + c x^2 + d x^3$ for some coefficients $(a, b, c, d) =: \theta$, and that all we have to do is estimate $\theta$. In this case, since after we have applied powers to our regressor $X_i$ the problem is still about recovering linear coefficients, we may use again linear regression:
$$ \widehat{\theta} := \text{arg}\min_{q \in \mathbb{R}^4} \sum_{i=1}^n \big( Y_i - (a + b X_i + c X_i^2 + d X_i^3) \big)^2 $$

Let us look how this goes with a code example:

In [ ]:
# DEFINE: function to generate nonlinear dataset
def generate_nlreg_data(n, range=[0,1], fun=np.sin, var=1, seed=202312):
    rng = np.random.default_rng(seed=seed)
    X = rng.uniform(low=range[0], high=range[1], size=n)
    E = rng.normal(loc=0, scale=var, size=n)
    Y = fun(X) + E
    return({'X': X, 'Y': Y, 'E': E})

In [ ]:
# DEFINE: polynomial regression
def polyreg(Y, X, p=3):
    n = len(Y.squeeze())
    Z = np.ones([n, 1])
    for i in range(1, p):
        Z = np.column_stack([Z, np.power(X, i)])
    theta_hat = np.linalg.solve(np.dot(Z.T, Z), np.dot(Z.T, Y))
    return(theta_hat)

In [ ]:
# PLOT: nonlinear regression data
def data_plot(n = 10):
    # generate data
    X_range = [0, 4]
    def f(x):
        return(np.sin(0.5 + x * 1.1) + 0.1)
    data = generate_nlreg_data(n, fun=f, range=X_range, var=0.1)

    # polynomial regression
    p = 3
    theta_hat = polyreg(data['Y'], data['X'], p=p)
    def polyf_hat(x):
        return(np.dot(np.array([x**i for i in range(0, p)]).T, theta_hat))

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(np.linspace(*X_range), f(np.linspace(*X_range)), c="0.6", zorder=1)
    ax.scatter(data['X'], data['Y'], zorder=2)
    ax.plot(np.linspace(*X_range), polyf_hat(np.linspace(*X_range)), c="C1", zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_xlim([-0.1, 4.1])
    ax.set_ylim([-1.3, 1.5])
    ax.set_title("Nonlinear Regression")
    ax.legend(["True Function", "Data", "Polynomial Estimate"])
    fig.canvas.draw()

interact(data_plot, n=(3, 100));

If our guess for the form of $f$ was good, we should intuitively see the following:

-  As the number of observations $n$ (the *sample size*) increases, the polynomial estimate (orange) should get close to the true function (gray).

- The error that we make over the interval of $X_i$, the range $[0, 4]$, the error we make should roughly be the same everywhere.

But these things do not seem to happen, even as $n \approx 100$. In fact, we can see that it is "settling" into a quite bad estimate!

### Bad Guesses and Nonlinear Estimation

Turns out (by looking at the code) that our guees if far from the truth. The true function in the above example is a *sinusoid*, which means our polynomial approximation does not a good job at capturing the underlying relationship between $Y_i$ and $X_i$, even forgoing the fact that noise $\epsilon_i$ is present.

Suppose now, however, that we did not make such a ill-informed guess, but that we had a good theoretical reason to say "The link between $Y$ and $X$ is sinusoidal, we just need to recover the right *frequency* ($\rho$), *shift* ($\tau$) and intercept ($\alpha$)." This is a easier problem now, on the surface, because we know the right "type" of nonlinearity, we just have to find a couple of scalars.

To achieve this, we set up the following problem:
$$ (\widehat{\alpha}, \widehat{\rho}, \widehat{\tau}) := \text{arg}\min_{(a, r, t) \in \mathbb{R}^2} \sum_{i=1}^n \big( Y_i - (a + \sin(t + r X_i)) \big)^2 .$$
But now comes the crutch: in our minimization problem, the coefficients $r$ and $t$ are *inside* the nonlinear function. This means that, in general, $\sum_{i=1}^n \big( Y_i - ( \alpha + \sin(t + r X_i)) \big)$ is not convex and so we cannot write $\widehat{\alpha}$, $\widehat{\rho}$ and $\widehat{\tau}$ in closed-form. Maybe, numerical optimization might even fail us.

Let us see with a code example:

In [ ]:
# DEFINE: sinusoidal regression
def sinusoidreg(Y, X):
    def sinusoid(x, a, r, t):
        return(a + np.sin(r * x + t))
    theta_hat, _ = opt.curve_fit(sinusoid, X, Y)
    return(theta_hat)

In [ ]:
# PLOT: sinusoid regression
def data_plot(n = 10):
    # generate data
    X_range = [0, 4]
    def f(x):
        return(np.sin(0.5 + x * 1.1) + 0.1)
    data = generate_nlreg_data(n, fun=f, range=X_range, var=0.1)

    # sinusoid regression
    theta_hat = sinusoidreg(data['Y'], data['X'])
    def sinusoid_hat(x):
        return(theta_hat[0] + np.sin(theta_hat[1] * x + theta_hat[2]))

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(np.linspace(*X_range), f(np.linspace(*X_range)), c="0.6", zorder=1)
    ax.scatter(data['X'], data['Y'], zorder=2)
    ax.plot(np.linspace(*X_range), sinusoid_hat(np.linspace(*X_range)), c="C4", zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_xlim([-0.1, 4.1])
    ax.set_ylim([-1.3, 1.5])
    ax.set_title("Nonlinear Regression")
    ax.legend(["True Function", "Data", "Sinusoid Estimate"])
    fig.canvas.draw()

interact(data_plot, n=(3, 100));

As we can see, the results of our "sinusoidal regression" are much better than those of the polynomial regression we prevously implemented.

But now, we ask, how do we do if we slightly change the true function?

In [ ]:
# PLOT: sinusoid regression
def data_plot(n = 10):
    # generate data
    X_range = [0, 4]
    def f(x):
        return(np.sin(0.5 + x * 1.4) + 0.1 + np.cos(x * 2.4) * 0.4)
    data = generate_nlreg_data(n, fun=f, range=X_range, var=0.1)

    # sinusoid regression
    theta_hat = sinusoidreg(data['Y'], data['X'])
    def sinusoid_hat(x):
        return(theta_hat[0] + np.sin(theta_hat[1] * x + theta_hat[2]))

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(np.linspace(*X_range), f(np.linspace(*X_range)), c="0.6", zorder=1)
    ax.scatter(data['X'], data['Y'], zorder=2)
    ax.plot(np.linspace(*X_range), sinusoid_hat(np.linspace(*X_range)), c="C4", zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_xlim([-0.1, 4.1])
    ax.set_ylim([-1.3, 1.5])
    ax.set_title("Nonlinear Regression")
    ax.legend(["True Function", "Data", "Sinusoid Estimate"])
    fig.canvas.draw()

interact(data_plot, n=(3, 100));

The true function now is combination of $\sin(\cdot)$ and $\cos(\cdot)$ functions, and, although periodic, it can not be easily approximated by our idea of fitting a $\sin(\cdot)$ with flexible frequency, shift and intercept parameters.

Now we might realize that there are two main paths:

1. We add $\cos(\cdot)$ to our roster of functions to include in the least-squares problem: one just needs to estimate more nonlinear coefficients.

2. Let us consider, instead, *linear combinations of a family of functions* that can approximate well **any function**.

The former option is often expensive and burdensome, but when done with care (in statistics, we often call this setup that of nonlinear *parametric* estimation) can still be very effective.

The latter option is the path that most modern machine learning methods - such as Neural Networks - take (when seen from the appropriate theoretical perspective).

We will now take this second path, and in fact work with a specific class of regression models that select functions *randomly*.

### Random-Weights Neural Networks

RWNN form
$$ h(x; W) = \sum_{j=1}^K W_j \sigma\left(\lang A_j , x \rang + \zeta_j \right) $$

$$ \widehat{W} := \text{arg}\min_{w \in \mathbb{R}^K} \sum_{i=1}^n \big( Y_i - h(X_i; W) \big)^2 + \lambda \Vert W \Vert_2^2 $$

Where $A_j$ is the $j^\text{th}$ row of the $A$ matrix. By letting $Z_i := \sigma(A X_i + \zeta)$ and $Z := (Z_1, \ldots, Z_n) \in \mathbb{R}^{K \times n}$, we can write down the solution to the previous ridge regression problem in closed form as
$$ \widehat{W} = (Z' Z + \lambda I)^{-1} Z' Y. $$
Moreover, the RWNN estimate is given by:
$$ \widehat{h}(x) := \sum_{k=1}^K \widehat{W}_j \sigma\left(\lang A_j, x \rang + \zeta_j \right)

To construct a RWNN, we just need to do the following:

1. Randomly sample coefficients in $A$ and $\zeta$ from an appropriate distribution.
2. Given input feature $X_i$, compute $Z_i := \sigma( A X_i + \zeta)$, the new transformed feature.
3. Perform ridge regression of $Y$ on $Z$.

Below we define function `rwnn()` which implements steps 2 and 3 given data, $A$, $\zeta$ and a ridge penalty.

In [ ]:
# DEFINE: random-weights neural network
def rwnn(Y, X, L, A, zeta, basefun):
    # transform the input features by multiplying with A and adding zeta
    Z = basefun(A * X[:,None] + zeta)
    print(Z.shape)
    N, K = Z.shape
    # ridge regression with penalty L
    # NOTE: we adjust here by N, the sample size, so that if we increase
    #       the sample size, the penalization remains comparable
    omega_hat = np.linalg.solve(np.dot(Z.T, Z) + (N*L)*np.eye(K), np.dot(Z.T, Y))
    return(omega_hat)

In the following example, we can take a look at how the RWNN performs as sample size and regularizations changes:

In [ ]:
# PLOT: rwnn regression
def data_plot(n = 10, Lambda = 1e-3):
    # generate data
    X_range = [0, 4]
    def f(x):
        return(np.sin(0.5 + x * 1.4) + 0.1 + np.cos(x * 2.4) * 0.4)
    data = generate_nlreg_data(n, fun=f, range=X_range, var=0.1)

    # random weights
    K = 20
    rng = np.random.default_rng(75871685)
    A = rng.normal(loc=0, scale=1, size=K)
    zeta = rng.uniform(low=0, high=4, size=K)
    
    # rwnn regression
    omega_hat = rwnn(data['Y'], data['X'], Lambda, A, zeta, np.tanh)
    def rwnn_hat(x):
        return(np.dot(np.tanh(A * x[:,None] + zeta), omega_hat))

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(np.linspace(*X_range), f(np.linspace(*X_range)), c="0.6", zorder=1)
    ax.scatter(data['X'], data['Y'], zorder=2)
    ax.plot(np.linspace(*X_range), rwnn_hat(np.linspace(*X_range)), c="C2", zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_xlim([-0.1, 4.1])
    ax.set_ylim([-1.3, 1.5])
    ax.set_title("Random-Weights-NN Regression")
    ax.legend(["True Function", "Data", "RWNN Estimate"])
    fig.canvas.draw()

interact(data_plot, n=(3, 100), Lambda=widgets.FloatLogSlider(min=-8, max=2, value=1e-6));

To better inspect the RWNN method, let us take a look "under the hood" and plot the functions that are being applied to $X$ when constructing the new features in $Z$:

In [ ]:
# PLOT: rwnn regression
def data_plot(p = 5, A_mean = 0, A_var = 1, zeta_range=[0., 4.]):
    # generate data
    X_range = [0, 4]
    def f(x):
        return(np.sin(0.5 + x * 1.4) + 0.1 + np.cos(x * 2.4) * 0.4)

    # random weights
    rng = np.random.default_rng(75871685)
    A = rng.normal(loc=A_mean, scale=A_var, size=p)
    zeta = rng.uniform(low=zeta_range[0], high=zeta_range[1], size=p)
    
    # rwnn regression
    def rwnn_featuremap(x, i):
        return(np.tanh(A[i] * x[:,None] + zeta[i]))

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(np.linspace(*X_range), f(np.linspace(*X_range)), c="0.6", zorder=1)
    for i in range(p):
        ax.plot(np.linspace(*X_range), rwnn_featuremap(np.linspace(*X_range), i), zorder=3)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_xlim([-0.1, 4.1])
    ax.set_ylim([-1.3, 1.5])
    ax.set_title("Random-Weights-NN Regression")
    ax.legend(["True Function"])
    fig.canvas.draw()

interact(data_plot, p=(3, 50), A_mean=(-10.0, 10.0), A_var=(0.01, 3), 
            zeta_range=widgets.FloatRangeSlider(value=[0, 4], min=-10.0, max=10.0));

Since both $A$ and $\zeta$ are random, what happens when we *resample* from their respective distribution and contruct a new RWNN estimate?

In the following plot, we do not contruct only one RWNN estimate, but instead use a number $p$ of different models. We can then plot both the *mean* estimate (green line) and the 95% frequency band (light green), which tells us how "dispersed" are the different RWNN estimates.

In [ ]:
# PLOT: rwnn redrawing
def data_plot(n = 10, Lambda = 1e-3, p = 10, draws = 5):
    # generate data
    X_range = [0, 4]
    def f(x):
        return(np.sin(0.5 + x * 1.4) + 0.1 + np.cos(x * 2.4) * 0.4)
    data = generate_nlreg_data(n, fun=f, range=X_range, var=0.1)

    X_linsp = np.linspace(*X_range)

    # rwnn 
    def rwnn_hat(x, A_, zeta_, omega_):
        return(np.dot(np.tanh(A_ * x[:,None] + zeta_), omega_))

    # redraw and fit
    rng = np.random.default_rng(75871685)

    fits = np.zeros([draws, len(X_linsp)])
    for b in range(draws):
        # random weights
        A_ = rng.normal(loc=0, scale=1, size=p)
        zeta_ = rng.uniform(low=0, high=4, size=p)
    
        # rwnn regression
        omega_hat = rwnn(data['Y'], data['X'], Lambda, A_, zeta_, np.tanh)
        fits[b,:] = rwnn_hat(X_linsp, A_, zeta_, omega_hat)

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(X_linsp, f(X_linsp), c="0.6", zorder=1)
    ax.scatter(data['X'], data['Y'], zorder=2)
    ax.plot(X_linsp, np.mean(fits, axis=0), c="C2", zorder=3)
    ax.fill_between(X_linsp, *np.quantile(fits, [0.05, 0.95], axis=0), color="C2", alpha=0.2)
    ax.set_axisbelow(True)
    ax.grid()
    ax.set_xlim([-0.1, 4.1])
    ax.set_ylim([-1.3, 1.5])
    ax.set_title("Nonlinear Regression")
    ax.legend(["True Function", "Data", "[Mean] RWNN Estimate", "95% Freq. Band"])
    fig.canvas.draw()

interact(data_plot, n=(3, 100), Lambda=widgets.FloatLogSlider(min=-8, max=2, value=1e-6),
            p=(3,100), draws=(1,50));